Copyright 2022 Google LLC. SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Experiment: RoboCodeGen Benchmark

This notebook is a part of the open-source code release associated with the paper:

[Code as Policies: Language Model Programs for Embodied Control](https://code-as-policies.github.io/)

This notebook gives the results corresponding to Table II in the paper which evaluates different code-gen approaches on a custom robotics-themed benchmark.

1) Please obtain an OpenAI API Key here:
https://openai.com/blog/openai-api/

2) Gain Codex access by joining the waitlist here:
https://openai.com/blog/openai-codex/

Note due to current rate limiting of the Codex API, this entire notebook may take a few hours to finish.

In [ ]:
openai_api_key = 'YOUR KEY HERE'

# Setup

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
! pip install numpy scipy shapely openai pygments RestrictedPython > /dev/null 2>&1

import openai
openai.api_key = openai_api_key

In [ ]:
from copy import copy
from time import sleep
from tqdm.auto import trange, tqdm

import ast
import astunparse

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter

def exec_safe(code_str, gvars, lvars):
  banned_phrases = ['import', '__']
  for phrase in banned_phrases:
    assert phrase not in code_str
  
  empty_fn = lambda *args, **kwargs: None
  custom_gvars = merge_dicts([
      gvars,
      {'exec': empty_fn, 'eval': empty_fn}
  ])
  exec(code_str, custom_gvars, lvars)

default_query_kwargs = {
    'engine': 'code-davinci-002',
    # 'engine': 'text-davinci-002',
    'max_tokens': 512,
    'temperature': 0,
    'frequency_penalty': 0
}

def lmp(base_prompt, query, stop_tokens=None, log=True, return_response=False, query_kwargs=None):
    new_prompt = f'{base_prompt}\n{query}'
    use_query_kwargs = copy(default_query_kwargs)
    if query_kwargs is not None:
      use_query_kwargs.update(query_kwargs)
    response = openai.Completion.create(
        prompt=new_prompt, stop=stop_tokens, **use_query_kwargs
    )['choices'][0]['text'].strip()

    if log:
      print(query)
      print(response)

    if return_response:
      return response

def lmp_fgen(prompt, f_name, f_sig, stop_tokens=['# define function:', '# example:'], recurse=False, 
             context_vars=None, bug_fix=False, log=True, return_src=False, query_kwargs=None, info=''):
    query = f'# define function: {f_sig}.'
    if info:
      query = f'{query}\n# info: {info}.'
    f_src = lmp(prompt, query, stop_tokens=stop_tokens, log=False, return_response=True, query_kwargs=query_kwargs)
    if bug_fix:
        sleep(6)
        f_src = openai.Edit.create(
          model='code-davinci-edit-001',
          input='# ' + f_src,
          temperature=0,
          instruction="Fix syntax errors. Keep same inputs and outputs. Only small changes. No comments.",
        )['choices'][0]['text'].strip()

    if context_vars is None:
        context_vars = {}
    gvars = context_vars
    lvars = {}

    f_success = True
    try:
      exec_safe(f_src, gvars, lvars)
      f = lvars[f_name]
    except Exception as e:
      print(e)
      f = lambda *args, **kargs: None   
      f_success = False 

    if recurse and f_success:
      # recursively define child_fs in the function body if needed
      f_def_body = astunparse.unparse(ast.parse(f_src).body[0].body)
      potential_child_fs, potential_child_f_sigs = {}, {}
      f_parser = FunctionParser(potential_child_fs, potential_child_f_sigs)
      f_parser.visit(ast.parse(f_def_body))
      for potential_child_f_name, potential_child_f_sig in potential_child_f_sigs.items():
        if potential_child_f_name in potential_child_fs:
          potential_child_fs[potential_child_f_name] = potential_child_f_sig

      child_fs, child_f_srcs = {}, {}
      for child_f_name, child_f_sig in potential_child_fs.items():
        all_vars = merge_dicts([context_vars, child_fs])
        if not var_exists(child_f_name, all_vars):
          child_f, child_f_src = lmp_fgen(
              prompt, child_f_name, child_f_sig, 
              stop_tokens=stop_tokens, 
              context_vars=all_vars, 
              bug_fix=bug_fix,
              log=False, 
              recurse=True,
              return_src=True,
              query_kwargs=query_kwargs
            )

          child_fs[child_f_name] = child_f
          child_f_srcs[child_f_name] = child_f_src

      if len(child_fs) > 0:
        # redefine parent f so newly created child_fs are in scope
        gvars = merge_dicts([context_vars, child_fs])
        lvars = {}
      
        exec_safe(f_src, gvars, lvars)
        
        f = lvars[f_name]

    if log:
        to_print = highlight(f'{query}\n{f_src}', PythonLexer(), TerminalFormatter())
        print(f'LMP FGEN created:\n\n{to_print}\n')

    if return_src:
        return f, f_src
    return f

def lmp_batch(base_prompt, cmds, stop_tokens=None, strip=False, batch_size=20, rate_limit_time=6, query_kwargs=None):
    prompts = [
      f'{base_prompt}\n{cmd}'
      for cmd in cmds
    ]

    use_query_kwargs = copy(default_query_kwargs)
    if query_kwargs is not None:
      use_query_kwargs.update(query_kwargs)

    responses = []
    for start_idx in trange(0, len(prompts), batch_size):
        end_idx = min(start_idx + batch_size, len(prompts))
        batch_prompts = prompts[start_idx : end_idx]

        raw_responses_batch = openai.Completion.create(
            prompt=batch_prompts, stop=stop_tokens, **use_query_kwargs
        )
        responses_batch = [
            r['text']
            for r in raw_responses_batch['choices']
        ]

        if strip:
            responses_batch = [response.strip() for response in responses_batch]
        
        responses.extend(responses_batch)

        if end_idx != len(prompts):
            sleep(rate_limit_time)

    return responses

def lmp_fgen_batch(prompt, f_names, f_sigs, stop_tokens=['# define function:', '# example:'], 
                   recurse=False, context_vars=None, bug_fix=False, log=True, query_kwargs=None,
                   rate_limit_time=6):
    queries = [f'# define function: {f_sig}.' for f_sig in f_sigs]
    f_srcs_list = lmp_batch(prompt, queries, stop_tokens=stop_tokens, query_kwargs=query_kwargs)
    
    if bug_fix:
        for idx in trange(len(f_srcs_list)):
          sleep(rate_limit_time)
          f_srcs_list[idx] = openai.Edit.create(
            model='code-davinci-edit-001',
            input='# ' + f_srcs_list[idx],
            temperature=0,
            instruction="Fix syntax errors. Keep same inputs and outputs. Only small changes. No comments.",
          )['choices'][0]['text'].strip()

    f_srcs = {f_name: f_src for f_name, f_src in zip(f_names, f_srcs_list)}

    fs = {}
    all_child_fs, all_child_f_srcs = {}, {}
    for f_name, f_sig, f_src in zip(f_names, f_sigs, f_srcs_list):
      if context_vars is None:
        context_vars = {}
      gvars = merge_dicts([context_vars, fs, all_child_fs])
      lvars = {}
    
      try:
        exec_safe(f_src, gvars, lvars)
        fs[f_name] = lvars[f_name]
      except Exception as e:
        print(f_name)
        print(f_src)
        print(e)
        fs[f_name] = lambda *args, **kwargs: None
        continue      

      # recursively define child_fs in the function body if needed
      if recurse:
        f_def_body = astunparse.unparse(ast.parse(f_src).body[0].body)
        potential_child_fs, potential_child_f_sigs = {}, {}
        f_parser = FunctionParser(potential_child_fs, potential_child_f_sigs)
        f_parser.visit(ast.parse(f_def_body))
        for potential_child_f_name, potential_child_f_sig in potential_child_f_sigs.items():
          if potential_child_f_name in potential_child_fs:
            potential_child_fs[potential_child_f_name] = potential_child_f_sig

        child_fs, child_f_srcs = {}, {}
        for child_f_name, child_f_sig in potential_child_fs.items():
          all_vars = merge_dicts([context_vars, fs, all_child_fs, child_fs])
          if not var_exists(child_f_name, all_vars):
            sleep(rate_limit_time)
            child_f, child_f_src = lmp_fgen(
                prompt, child_f_name, child_f_sig, 
                stop_tokens=stop_tokens, 
                context_vars=all_vars, 
                bug_fix=bug_fix, 
                log=False,
                recurse=True,
                return_src=True,
                query_kwargs=query_kwargs
              )

            child_fs[child_f_name] = child_f
            child_f_srcs[child_f_name] = child_f_src

        if len(child_fs) > 0:
          # redefine parent f so newly created child_fs are in scope
          gvars = merge_dicts([context_vars, fs, all_child_fs, child_fs])
          lvars = {}
        
          exec_safe(f_src, gvars, lvars)
          
          fs[f_name] = lvars[f_name]
          all_child_fs.update(child_fs)
          all_child_f_srcs.update(child_f_srcs)

    if log:
      for query, f_src in zip(queries, f_srcs):
        to_print = highlight(f'{query}\n{f_src}', PythonLexer(), TerminalFormatter())
        print(f'LMP FGEN created:\n\n{to_print}\n')

    all_fs = merge_dicts([fs, all_child_fs])
    all_f_srcs = merge_dicts([f_srcs, all_child_f_srcs])
    f_gens = {
        f_name: {
            'f': f,
            'f_src': all_f_srcs[f_name]
        }
        for f_name, f in all_fs.items()
    }

    return f_gens

class FunctionParser(ast.NodeTransformer):

    def __init__(self, fs, f_assigns):
      super().__init__()
      self._fs = fs
      self._f_assigns = f_assigns

    def visit_Call(self, node):
        self.generic_visit(node)
        if isinstance(node.func, ast.Name):
            f_sig = astunparse.unparse(node).strip()
            f_name = astunparse.unparse(node.func).strip()
            self._fs[f_name] = f_sig
        return node

    def visit_Assign(self, node):
        self.generic_visit(node)
        if isinstance(node.value, ast.Call):
            assign_str = astunparse.unparse(node).strip()
            f_name = astunparse.unparse(node.value.func).strip()
            self._f_assigns[f_name] = assign_str
        return node

def var_exists(name, all_vars):
    try:
        eval(name, all_vars)
    except:
        exists = False
    else:
        exists = True
    return exists

def merge_dicts(dicts):
    return {
        k : v 
        for d in dicts
        for k, v in d.items()
    }

def eval_test(f_sol, f_gen, test_input_args, test_equiv=None):
  if test_equiv is None:
    test_equiv = lambda out_f_sol, out_f_gen: np.allclose(out_f_sol, out_f_gen)
  
  test_success = True
  for test_input_arg in test_input_args:
    out_correct = f_sol(*test_input_arg)
    out_f_gen = f_gen(*test_input_arg)

    if not test_equiv(out_correct, out_f_gen):
      test_success = False
      break
  return test_success

def solve_problems(problems, prompt, context_vars, log=False, recurse=False, bug_fix=False, query_kwargs=None):
  f_names = [problem['f_name'] for problem in problems]
  f_sigs = [problem['f_sig'] for problem in problems]
  f_gens = lmp_fgen_batch(
      prompt, f_names, f_sigs, 
      context_vars=context_vars, bug_fix=bug_fix, recurse=recurse, 
      log=log, query_kwargs=query_kwargs
    )
  return f_gens

def eval_problems(problems, f_gens):
  results = []

  for problem in problems:
    success = False
    info = ''
    try:
      success = eval_test(problem['f_sol'], f_gens[problem['f_name']]['f'], problem['test_input_args'], problem['test_equiv'])
    except Exception as e:
      info = str(e)

    results.append({
        'f_name': problem['f_name'],
        'problem': problem,
        'f_gen': f_gens[problem['f_name']],
        'success': success,
        'info': info
    })

  n_successes = np.sum([r['success'] for r in results])
  success_rate = n_successes / len(problems)
  print(f'Success rate: {n_successes}/{len(problems)} = {success_rate:.2f}')

  return results

# Robot Code-Gen Benchmark

## Prompt

In [ ]:
prompt_f_gen = '''
import numpy as np
from shapely.geometry import *
from shapely.affinity import *
from utils import get_obj_outer_pts_np

# define function: total = get_total(xs=numbers).
def get_total(xs):
    return np.sum(xs)

# define function: y = eval_line(x, slope, y_intercept=0).
def eval_line(x, slope, y_intercept):
    return x * slope + y_intercept

# define function: pt_np = move_pt_left(pt_np, dist).
def move_pt_left(pt_np, dist):
    delta = np.array([-dist, 0])
    return translate_pt_np(pt_np, delta=delta)

# define function: pt_np = move_pt_up(pt_np, dist).
def move_pt_up(pt_np, dist):
    delta = np.array([0, dist])
    return translate_pt_np(pt_np, delta=delta)

# define function line = make_line_by_length(length=x).
def make_line_by_length(length):
  start = np.array([0, 0])
  end = np.array([length, 0])
  line = make_line(start=start, end=end)
  return line

# define function: line = make_vertical_line_by_length(length=x).
def make_vertical_line_by_length(length):
  line = make_line_by_length(length)
  vertical_line = rotate(line, 90)
  return vertical_line

# define function: pt = interpolate_line(line, t=0.5).
def interpolate_line(line, t):
  pt = line.interpolate(t, normalized=True)
  return np.array(pt.coords[0])

# example: scale a line by 2 around the centroid.
line = make_line_by_length(1)
new_shape = scale(line, xfact=2, yfact=2, origin='centroid')

# example: rotate a point around origin by 45 degrees.
pt = Point([1,1])
new_pt = rotate(pt, 45, origin=[0, 0])

# example: getting object points of object0.
pts_np = get_obj_outer_pts_np('object0')
'''.strip()

prompt_f_gen_flat = '''
import numpy as np
from shapely.geometry import *
from shapely.affinity import *
from utils import get_obj_outer_pts_np

# define function: total = get_total(xs=numbers).
def get_total(xs):
    return np.sum(xs)

# define function: y = eval_line(x, slope, y_intercept=0).
def eval_line(x, slope, y_intercept):
    return x * slope + y_intercept

# define function: pt_np = move_pt_left(pt_np, dist).
def move_pt_left(pt_np, dist):
    return pt_np + [-dist, 0]

# define function: pt_np = move_pt_up(pt_np, dist).
def move_pt_up(pt_np, dist):
    return pt_np + [0, dist]

# define function line = make_line_by_length(length=x).
def make_line_by_length(length):
  line = LineString([[0, 0], [length, 0]])
  return line

# define function: line = make_vertical_line_by_length(length=x).
def make_vertical_line_by_length(length):
  line = make_line_by_length(length)
  vertical_line = rotate(line, 90)
  return vertical_line

# define function: pt = interpolate_line(line, t=0.5).
def interpolate_line(line, t):
  pt = line.interpolate(t, normalized=True)
  return np.array(pt.coords[0])

# example: scale a line by 2.
line = make_line_by_length(1)
new_shape = scale(line, xfact=2, yfact=2)

# example: rotate a point around origin by 45 degrees.
pt = Point([1,1])
new_pt = rotate(pt, 45, origin=[0, 0])

# example: getting object points of object0.
pts_np = get_obj_outer_pts_np('object0')
'''.strip()

prompt_f_gen_exec = '''
import numpy as np
from shapely.geometry import *
from shapely.affinity import *

# define function: total = get_total(xs=numbers).
def get_total(xs):
    return np.sum(xs)

# define function: y = eval_line(x, slope, y_intercept=0).
def eval_line(x, slope, y_intercept):
    return x * slope + y_intercept

# define function: pt = move_pt_left(pt, dist).
def move_pt_left(pt, dist):
    return pt + [-dist, 0]

# define function: pt = move_pt_up(pt, dist).
def move_pt_up(pt, dist):
    return pt + [0, dist]

# define function line = make_line_by_length(length=x).
def make_line_by_length(length):
  line = LineString([[0, 0], [length, 0]])
  return line

# define function: line = make_vertical_line_by_length(length=x).
def make_vertical_line_by_length(length):
  line = make_line_by_length(length)
  vertical_line = rotate(line, 90)
  return vertical_line

# define function: pt = interpolate_line(line, t=0.5).
def interpolate_line(line, t):
  pt = line.interpolate(t, normalized=True)
  return np.array(pt.coords[0])
'''.strip()

context_vars = {}
exec(prompt_f_gen_exec, context_vars)

## Problems

In [ ]:
import numpy as np
import shapely
from shapely.geometry import *
from shapely.affinity import *
from operator import eq

def rotate_pts_around_pts_center_np(pts_np, angle_deg):
  angle_rad = np.deg2rad(angle_deg)
  centroid = np.mean(pts_np, axis=0)
  R = np.array([
      [np.cos(angle_rad), -np.sin(angle_rad)], 
      [np.sin(angle_rad), np.cos(angle_rad)]
  ])
  new_pts = (pts_np - centroid) @ R.T + centroid
  return new_pts

def scale_pts_around_centroid_np(pts_np, scale_x=1.5, scale_y=1.5):
    centroid = np.mean(pts_np, axis=0)
    new_pts = pts_np - centroid
    new_pts[:, 0] = new_pts[:, 0] * scale_x
    new_pts[:, 1] = new_pts[:, 1] * scale_y
    new_pts = new_pts + centroid
    return new_pts

problems_np = [
  {
    'f_name': 'get_top_most_idx',
    'f_sig': 'idx = get_top_most_idx(points_np)',
    'f_sol': lambda points_np: np.argmax(points_np[:, 1]),
    'test_input_args': [(np.random.random((10, 2)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_bottom_most_idx',
    'f_sig': 'idx = get_bottom_most_idx(points_np)',
    'f_sol': lambda points_np: np.argmin(points_np[:, 1]),
    'test_input_args': [(np.random.random((10, 2)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_left_most_idx',
    'f_sig': 'idx = get_left_most_idx(points_np)',
    'f_sol': lambda points_np: np.argmin(points_np[:, 0]),
    'test_input_args': [(np.random.random((10, 2)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_right_most_idx',
    'f_sig': 'idx = get_right_most_idx(points_np)',
    'f_sol': lambda points_np: np.argmax(points_np[:, 0]),
    'test_input_args': [(np.random.random((10, 2)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_farthest_idx',
    'f_sig': 'idx = get_farthest_idx(points_np, point_np)',
    'f_sol': lambda points_np, point_np: np.argmax(np.linalg.norm(points_np - point_np, axis=1)),
    'test_input_args': [(np.random.random((10, 2)), np.random.random(2)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_closest_idx',
    'f_sig': 'idx = get_closest_idx(points_np, point_np)',
    'f_sol': lambda points_np, point_np: np.argmin(np.linalg.norm(points_np - point_np, axis=1)),
    'test_input_args': [(np.random.random((10, 2)), np.random.random(2)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_bbox_xyxy_area',
    'f_sig': 'area = get_bbox_xyxy_area(bbox_xyxy)',
    'f_sol': lambda bbox_xyxy: (bbox_xyxy[2] - bbox_xyxy[0]) * (bbox_xyxy[3] - bbox_xyxy[1]),
    'test_input_args': [(np.random.random(4) + [0, 0, 1, 1],) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'bbox_xyxy_contains_pt',
    'f_sig': 'contains = bbox_xyxy_contains_pt(bbox_xyxy)',
    'f_sol': lambda bbox_xyxy, pt: pt[0] >= bbox_xyxy[0] and pt[0] <= bbox_xyxy[2] and pt[1] >= bbox_xyxy[1] and pt[1] <= bbox_xyxy[3],
    'test_input_args': [(np.random.random(4) * [-1, -1, 1, 1], np.random.random(2) *2 - 1) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'interpolate_pts_np',
    'f_sig': 'pts = interpolate_pts_np(start, end, n)',
    'f_sol': lambda start, end, n: np.linspace(start, end, n),
    'test_input_args': [(np.random.random(2), np.random.random(2), 3) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'reverse_pts',
    'f_sig': 'pts = reverse_pts(pts_np)',
    'f_sol': lambda pts_np: pts_np[::-1],
    'test_input_args': [(np.random.random((10, 2)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'normalize_vector',
    'f_sig': 'normalized_vector = normalize_vector(vector)',
    'f_sol': lambda vector: vector / np.linalg.norm(vector),
    'test_input_args': [(np.random.random(np.random.randint(1, 10)), ) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'translate_pts_np',
    'f_sig': 'new_pts_np = translate_pts_np(pts_np, delta_np)',
    'f_sol': lambda pts_np, delta_np: pts_np + delta_np,
    'test_input_args': [(np.random.random((10, 2)), np.random.random(2)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'rotate_pts_around_pts_center_np',
    'f_sig': 'new_pts_np = rotate_pts_around_pts_center_np(pts_np, angle_deg)',
    'f_sol': rotate_pts_around_pts_center_np,
    'test_input_args': [(np.random.random((10, 2)), np.random.random()) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'scale_pts_around_centroid_np',
    'f_sig': 'new_pts_np = scale_pts_around_centroid_np(pts_np, scale_x=1.5, scale_y=1.5)',
    'f_sol': scale_pts_around_centroid_np,
    'test_input_args': [(np.random.random((10, 2)), np.random.random(), np.random.random()) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'fit_2d_np_polys',
    'f_sig': '(poly_x_coeffs, poly_y_coeffs) = fit_2d_np_polys(ts, pts_2d_np, deg)',
    'f_sol': lambda ts, pts_2d_np, deg: (np.polyfit(ts, pts_2d_np[:, 0], deg), np.polyfit(ts, pts_2d_np[:, 1], deg)),
    'test_input_args': [(np.linspace(0, 10, 10), np.random.random((10, 2)), np.random.randint(3, 6)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'evaluate_pts_2d_from_poly_coeffs',
    'f_sig': 'pts_2d_np = evaluate_pts_2d_from_poly_coeffs(poly_x_coeffs, poly_y_coeffs, ts)',
    'f_sol': lambda poly_x_coeffs, poly_y_coeffs, ts: np.stack([np.polyval(poly_x_coeffs, ts), np.polyval(poly_y_coeffs, ts)], axis=1),
    'test_input_args': [(np.random.random(5), np.random.random(5), np.linspace(0, 10, 10)) for _ in range(5)],
    'test_equiv': np.allclose
  }
]

problems_ctrl = [
  {
    'f_name': 'pd_control',
    'f_sig': 'u = pd_control(x_curr, x_goal, x_dot, Kp, Kv)',
    'f_sol': lambda x_curr, x_goal, x_dot, Kp, Kv: Kp * (x_goal - x_curr) - Kv * x_dot,
    'test_input_args': [(
      np.random.random(2), np.random.random(2), np.random.random(2), np.random.random(), np.random.random()
      ) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'end_effector_impedance_control',
    'f_sig': 'tau = end_effector_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J)',
    'f_sol': lambda x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J: J.T @ (K_x_mat @ (x_goal - x_curr) - D_x_mat @ x_dot),
    'test_input_args': [(
      np.random.random(3), np.random.random(3), np.random.random(3), 
      np.diag(np.random.random(3)), np.diag(np.random.random(3)), 
      np.random.random((3, 6))
      ) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'is_discrete_system_stable',
    'f_sig': 'is_stable = is_discrete_system_stable(A_mat)',
    'f_sol': lambda A_mat: np.all(np.abs(np.linalg.eigvals(A_mat)) < 1),
    'test_input_args': [(np.random.random((5, 5)),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'is_closed_loop_discrete_system_stable',
    'f_sig': 'is_stable = is_closed_loop_discrete_system_stable(A_mat, B_mat, K_mat)',
    'f_sol': lambda A_mat, B_mat, K_mat: np.all(np.abs(np.linalg.eigvals(A_mat - B_mat @ K_mat)) < 1),
    'test_input_args': [(np.random.random((5, 5)), np.random.random((5, 3)), np.random.random((3, 5))) for _ in range(5)],
    'test_equiv': np.allclose
  },
]

def get_direction_orthogonal_to_line_to_point(line, point):
  # get the line's direction.
  direction = np.array(line.coords[1]) - np.array(line.coords[0])
  direction = direction / np.linalg.norm(direction)
  # get the orthogonal direction.
  orthogonal_direction = np.array([-direction[1], direction[0]])
  # get the point's direction.
  point_direction = np.array(point) - np.array(line.coords[0])
  # get the sign of the point's direction.
  sign = np.sign(np.dot(point_direction, orthogonal_direction))
  # get the orthogonal direction from the line to the point.
  orthogonal_direction_from_line_to_point = sign * orthogonal_direction
  return orthogonal_direction_from_line_to_point

def get_direction_orthogonal_to_line(line):
  direction = np.array(line.coords[1]) - np.array(line.coords[0])
  direction = direction / np.linalg.norm(direction)
  direction = np.array([direction[1], -direction[0]])
  return direction

shape_eq = lambda f_sol_out, f_gen_out: np.allclose(f_sol_out.union(f_gen_out).area, f_sol_out.area, f_gen_out.area)

make_rectangle = lambda width, height, center: box(center[0] - width / 2, center[1] - height / 2, center[0] + width / 2, center[1] + height / 2)

problems_shapely = [
  {
    'f_name': 'get_closest_point_on_line_to_point',
    'f_sig': 'point_np = get_closest_point_on_line_to_point(line, pt_np)',
    'f_sol': lambda line, pt_np: np.array(line.interpolate(line.project(Point(pt_np))).coords[0]),
    'test_input_args': [(LineString(np.random.random((2, 2))), np.random.random(2)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_direction_orthogonal_to_line',
    'f_sig': 'direction = get_direction_orthogonal_to_line(line)',
    'f_sol': get_direction_orthogonal_to_line,
    'test_input_args': [(LineString(np.random.random((2, 2))),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_direction_orthogonal_to_line_to_point',
    'f_sig': 'direction = get_direction_orthogonal_to_line_to_point(line, pt_np)',
    'f_sol': get_direction_orthogonal_to_line_to_point,
    'test_input_args': [(LineString(np.random.random((2, 2))), np.random.random(2)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'get_points_from_polygon',
    'f_sig': 'points_np = get_points_from_polygon(polygon)',
    'f_sol': lambda polygon: np.array(polygon.exterior.coords),
    'test_input_args': [(Polygon(np.random.random((6, 2))),) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'interpolate_pts_along_exterior',
    'f_sig': 'pts_coords = interpolate_pts_along_exterior(exterior=shape.exterior, n=5)',
    'f_sol': lambda exterior, n: [exterior.interpolate(i / (n - 1), normalized=True).coords[0] for i in range(n)],
    'test_input_args': [(Polygon(np.random.random((6, 2))).exterior, np.random.randint(1, 100)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'interpolate_pts_on_line',
    'f_sig': 'pts_coords = interpolate_pts_on_line(line, n)',
    'f_sol': lambda exterior, n: [exterior.interpolate(i / (n - 1), normalized=True).coords[0] for i in range(n)],
    'test_input_args': [(LineString(np.random.random((2, 2))), np.random.randint(2, 100)) for _ in range(5)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'make_line',
    'f_sig': 'line = make_line(start_pt_np, end_pt_np)',
    'f_sol': lambda start_pt_np, end_pt_np: LineString([start_pt_np, end_pt_np]),
    'test_input_args': [(np.random.random(2), np.random.random(2)) for _ in range(5)],
    'test_equiv': eq
  },
  {
    'f_name': 'make_circle',
    'f_sig': 'circle = make_circle(radius, center)',
    'f_sol': lambda radius, center: Point(center).buffer(radius),
    'test_input_args': [(np.random.random(), np.random.random(2)) for _ in range(5)],
    'test_equiv': shape_eq
  },
  {
    'f_name': 'make_rectangle',
    'f_sig': 'rectangle = make_rectangle(width, height, center)',
    'f_sol': make_rectangle,
    'test_input_args': [(np.random.random(), np.random.random(), np.random.random(2)) for _ in range(5)],
    'test_equiv': shape_eq
  },
  {
    'f_name': 'make_ellipse',
    'f_sig': 'ellipse = make_ellipse(center, major_axis, minor_axis)',
    'f_sol': lambda center, major_axis, minor_axis: scale(Point(center).buffer(1.0), xfact=major_axis, yfact=minor_axis),
    'test_input_args': [(np.random.random(2), np.random.random(), np.random.random()) for _ in range(5)],
    'test_equiv': shape_eq
  },
]

obj_names = ['block0', 'block1', 'block2', 'block3', 'block4', 'block5']
obj_positions = np.array([
    [0, 0],
    [0.1, 0],
    [0.5, 0.5],
    [0, 0],
    [0.6, 0.6],
    [0, 0.9],
    [0.4, 0.4]
])
sizes = [0.15, 0.15, 0.1, 0.2, 0.1, 0.05, 0.1]
obj_boxes = [make_rectangle(size, size, pos) for pos, size in zip(obj_positions, sizes)]
obj_outer_pts_np_map = {
    obj_name: np.array(obj_box.exterior.coords)
    for obj_name, obj_box in zip(obj_names, obj_boxes)
}
get_obj_outer_pts_np = lambda obj_name: obj_outer_pts_np_map[obj_name]
obj_names0 = obj_names[:3]
obj_names1 = obj_names[3:]

def get_obj_shape(obj_name):
    pts = get_obj_outer_pts_np(obj_name)
    obj_shape = Polygon(pts)
    return obj_shape

def obj_shape_does_not_contain_others(obj_name, other_obj_names):
    obj_shape = get_obj_shape(obj_name)
    for other_obj_name in other_obj_names:
        other_obj_shape = get_obj_shape(other_obj_name)
        if obj_shape.contains(other_obj_shape):
            return False
    return True

def is_obj0_bigger_than_obj1(obj0_name, obj1_name):
    obj0_pts = get_obj_outer_pts_np(obj0_name)
    obj1_pts = get_obj_outer_pts_np(obj1_name)
    obj0_area = get_area(obj0_pts)
    obj1_area = get_area(obj1_pts)
    return obj0_area > obj1_area

def get_area(pts):
    polygon = Polygon(pts)
    return polygon.area

def get_one_bbox_xyxy_of_all_objs(all_obj_names):
    all_pts_np = []
    for obj_name in all_obj_names:
        pts_np = get_obj_outer_pts_np(obj_name)
        all_pts_np.append(pts_np)
    all_pts_np = np.concatenate(all_pts_np, axis=0)
    bbox_xyxy = get_bbox_xyxy_of_pts_np(all_pts_np)
    return bbox_xyxy

def get_bbox_xyxy_of_pts_np(all_pts_np):
    x_min = np.min(all_pts_np[:, 0])
    x_max = np.max(all_pts_np[:, 0])
    y_min = np.min(all_pts_np[:, 1])
    y_max = np.max(all_pts_np[:, 1])
    bbox_xyxy = np.array([x_min, y_min, x_max, y_max])
    return bbox_xyxy

def are_obj_centers_close(obj_names, threshold):
    centers = [get_obj_center(obj_name) for obj_name in obj_names]
    distances = [get_distance(centers[0], center) for center in centers[1:]]
    return all([distance < threshold for distance in distances])

def get_distance(pt0, pt1):
    return np.linalg.norm(pt0 - pt1)

def get_obj_center(obj_name):
    pts_np = get_obj_outer_pts_np(obj_name)
    center = np.mean(pts_np, axis=0)
    return center

def get_name_of_biggest_obj(obj_names):
    biggest_obj_name = None
    biggest_obj_area = 0
    for obj_name in obj_names:
        obj_area = get_obj_area(obj_name)
        if obj_area > biggest_obj_area:
            biggest_obj_area = obj_area
            biggest_obj_name = obj_name
    return biggest_obj_name

def get_obj_area(obj_name):
    pts_np = get_obj_outer_pts_np(obj_name)
    polygon = Polygon(pts_np)
    return polygon.area

def do_any_object_shapes_intersect_with_each_other(obj_names):
    for i in range(len(obj_names)):
        for j in range(i+1, len(obj_names)):
            obj_name_i = obj_names[i]
            obj_name_j = obj_names[j]
            obj_shape_i = get_obj_shape(obj_name_i)
            obj_shape_j = get_obj_shape(obj_name_j)
            if obj_shape_i.intersects(obj_shape_j):
                return True
    return False

def get_name_of_obj_with_min_dist_to_pt(pt_np, obj_names):
    min_dist = np.inf
    min_dist_obj_name = None
    for obj_name in obj_names:
        obj_pts_np = get_obj_outer_pts_np(obj_name)
        dist = get_min_dist_between_pts_np_and_pts_np(pts_np=obj_pts_np, pt_np=pt_np)
        if dist < min_dist:
            min_dist = dist
            min_dist_obj_name = obj_name
    return min_dist_obj_name

def get_min_dist_between_pts_np_and_pts_np(pts_np, pt_np):
    dists = np.linalg.norm(pts_np - pt_np, axis=1)
    return np.min(dists)

problems_api = [
  {
    'f_name': 'obj_shape_does_not_contain_others',
    'f_sig': 'ret_val = obj_shape_does_not_contain_others(obj_name, other_obj_names)',
    'f_sol': obj_shape_does_not_contain_others,
    'test_input_args': [(obj_name0, obj_names1) for obj_name0 in obj_names0],
    'test_equiv': eq
  },
  {
    'f_name': 'is_obj0_bigger_than_obj1',
    'f_sig': 'ret_val = is_obj0_bigger_than_obj1(obj0_name, obj1_name)',
    'f_sol': is_obj0_bigger_than_obj1,
    'test_input_args': [(np.random.choice(obj_names0), np.random.choice(obj_names1)) for _ in range(5)],
    'test_equiv': eq
  },
  {
    'f_name': 'get_one_bbox_xyxy_of_all_objs',
    'f_sig': 'bbox_xyxy = get_one_bbox_xyxy_of_all_objs(all_obj_names)',
    'f_sol': get_one_bbox_xyxy_of_all_objs,
    'test_input_args': [(obj_names0,), (obj_names1,)],
    'test_equiv': np.allclose
  },
  {
    'f_name': 'are_obj_centers_close',
    'f_sig': 'ret_val = are_obj_centers_close(obj_names, threshold)',
    'f_sol': are_obj_centers_close,
    'test_input_args': [(obj_names0, np.random.random()), (obj_names1, np.random.random())],
    'test_equiv': eq
  },
  {
    'f_name': 'get_name_of_biggest_obj',
    'f_sig': 'obj_name = get_name_of_biggest_obj(obj_names)',
    'f_sol': get_name_of_biggest_obj,
    'test_input_args': [(obj_names0,), (obj_names1,)],
    'test_equiv': eq
  },
  {
    'f_name': 'do_any_object_shapes_intersect_with_each_other',
    'f_sig': 'ret_val = do_any_object_shapes_intersect_with_each_other(obj_names)',
    'f_sol': do_any_object_shapes_intersect_with_each_other,
    'test_input_args': [(obj_names0,), (obj_names1,)],
    'test_equiv': eq
  },
  {
    'f_name': 'get_name_of_obj_with_min_dist_to_pt',
    'f_sig': 'obj_name = get_name_of_obj_with_min_dist_to_pt(pt_np, obj_names)',
    'f_sol': get_name_of_obj_with_min_dist_to_pt,
    'test_input_args': [(np.random.random(2), obj_names0), (np.random.random(2), obj_names1)],
    'test_equiv': eq
  },
]

all_problems = problems_np + problems_ctrl + problems_shapely + problems_api
print('n problems:', len(all_problems))

context_vars['get_obj_outer_pts_np'] = get_obj_outer_pts_np

n problems: 37


In [ ]:
for p in all_problems:
  print(p['f_sig'])

idx = get_top_most_idx(points_np)
idx = get_bottom_most_idx(points_np)
idx = get_left_most_idx(points_np)
idx = get_right_most_idx(points_np)
idx = get_farthest_idx(points_np, point_np)
idx = get_closest_idx(points_np, point_np)
area = get_bbox_xyxy_area(bbox_xyxy)
contains = bbox_xyxy_contains_pt(bbox_xyxy)
pts = interpolate_pts_np(start, end, n)
pts = reverse_pts(pts_np)
normalized_vector = normalize_vector(vector)
new_pts_np = translate_pts_np(pts_np, delta_np)
new_pts_np = rotate_pts_around_pts_center_np(pts_np, angle_deg)
new_pts_np = scale_pts_around_centroid_np(pts_np, scale_x=1.5, scale_y=1.5)
(poly_x_coeffs, poly_y_coeffs) = fit_2d_np_polys(ts, pts_2d_np, deg)
pts_2d_np = evaluate_pts_2d_from_poly_coeffs(poly_x_coeffs, poly_y_coeffs, ts)
u = pd_control(x_curr, x_goal, x_dot, Kp, Kv)
tau = end_effector_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J)
is_stable = is_discrete_system_stable(A_mat)
is_stable = is_closed_loop_discrete_system_stable(A_mat, B_mat, K_mat)


## Evals

### code-davinci-002

In [ ]:
print('Codex | Hierarchical Code-Gen | Hierarchical Prompt')

f_gens_codex_hc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=True, query_kwargs={'engine': 'code-davinci-002'})

results_codex_hc_hp = eval_problems(all_problems, f_gens_codex_hc_hp)
for r in results_codex_hc_hp:
  print(int(r['success']))

# failures_codex_hc_hp = [r for r in results_codex_hc_hp if not r['success']]
# for failure in failures_codex_hc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

Codex | Hierarchical Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

ERROR:shapely.geos:third argument of GEOSProject_r must be Point


Success rate: 36/37 = 0.97
get_direction_orthogonal_to_line_to_point  
def get_direction_orthogonal_to_line_to_point(line, pt_np):
  # get the line from the origin to the point.
  line_to_pt = make_line(start=np.array([0, 0]), end=pt_np)
  # get the angle between the two lines.
  angle = get_angle_between_lines(line, line_to_pt)
  # get the direction orthogonal to the line.
  direction = get_direction_orthogonal_to_line(line)
  # rotate the direction by the angle.
  direction = rotate_direction(direction, angle)
  return direction




In [ ]:
print('Codex | Hierarchical Code-Gen | Flat Prompt')

f_gens_codex_hc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=True, query_kwargs={'engine': 'code-davinci-002'})

results_codex_hc_fp = eval_problems(all_problems, f_gens_codex_hc_fp)
failures_codex_hc_fp = [r for r in results_codex_hc_fp if not r['success']]
for failure in failures_codex_hc_fp:
  print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

Codex | Hierarchical Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

Success rate: 32/37 = 0.86
is_closed_loop_discrete_system_stable operands could not be broadcast together with shapes (5,3) (3,5)  
def is_closed_loop_discrete_system_stable(A_mat, B_mat, K_mat):
    # check if the system is stable.
    eig_vals, eig_vecs = np.linalg.eig(A_mat - B_mat * K_mat)
    is_stable = np.all(np.abs(eig_vals) < 1)
    return is_stable


get_direction_orthogonal_to_line_to_point  
def get_direction_orthogonal_to_line_to_point(line, pt_np):
  pt_np = np.array(pt_np)
  line_np = np.array(line.coords)
  line_vec = line_np[1] - line_np[0]
  pt_vec = pt_np - line_np[0]
  direction = np.cross(line_vec, pt_vec)
  return direction


obj_shape_does_not_contain_others 'numpy.ndarray' object has no attribute 'contains' 
def obj_shape_does_not_contain_others(obj_name, other_obj_names):
    obj_shape = get_obj_outer_pts_np(obj_name)
    for other_obj_name in other_obj_names:
        other_obj_shape = get_obj_outer_pts_np(other_obj_name)
        if obj_shape.contains(other_obj

/usr/local/lib/python3.7/dist-packages/shapely/affinity.py:109: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if origin == 'center':
/usr/local/lib/python3.7/dist-packages/shapely/affinity.py:113: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif origin == 'centroid':


In [ ]:
print('Codex | Flat Code-Gen | Hierarchical Prompt')

f_gens_codex_fc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=False, query_kwargs={'engine': 'code-davinci-002'})

results_codex_fc_hp = eval_problems(all_problems, f_gens_codex_fc_hp)

for r in results_codex_fc_hp:
  print(int(r['success']))

# failures_codex_fc_hp = [r for r in results_codex_fc_hp if not r['success']]
# for failure in failures_codex_fc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

Codex | Flat Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

is_closed_loop_discrete_system_stable

def is_closed_loop_discrete_system_stable(A_mat, B_mat, K_mat):
    # check if the system is stable.
    # A_mat: state matrix.
    # B_mat: input matrix.
    # K_mat: feedback gain matrix.
    # return: is_stable: boolean.
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    # reference: https://www.youtube.com/watch?v=Q_QX_QZQZQk
    #

In [ ]:
print('Codex | Flat Code-Gen | Flat Prompt')

f_gens_codex_fc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=False, query_kwargs={'engine': 'code-davinci-002'})

results_codex_fc_fp = eval_problems(all_problems, f_gens_codex_fc_fp)

for r in results_codex_fc_fp:
  print(int(r['success']))

# failures_codex_fc_fp = [r for r in results_codex_fc_fp if not r['success']]
# for failure in failures_codex_fc_fp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

Codex | Flat Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

ERROR:shapely.geos:third argument of GEOSProject_r must be Point


Success rate: 30/37 = 0.81
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
1
0
0
0
1


<string>:6: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.


### code-cushman-001

In [ ]:
print('cushman | Hierarchical Code-Gen | Hierarchical Prompt')

f_gens_cushman_hc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=True, query_kwargs={'engine': 'code-cushman-001'})

results_cushman_hc_hp = eval_problems(all_problems, f_gens_cushman_hc_hp)
failures_cushman_hc_hp = [r for r in results_cushman_hc_hp if not r['success']]
for failure in failures_cushman_hc_hp:
  print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

cushman | Hierarchical Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

name 'Ellipse' is not defined
Success rate: 21/37 = 0.57
bbox_xyxy_contains_pt bbox_xyxy_contains_pt() takes 1 positional argument but 2 were given 
def bbox_xyxy_contains_pt(bbox_xyxy):
    xmin, ymin, xmax, ymax = bbox_xyxy
    return lambda pt: xmin <= pt[0] <= xmax and ymin <= pt[1] <= ymax


interpolate_pts_np  
def interpolate_pts_np(start, end, n):
    pts = []
    for i in range(n):
        t = i / n
        pt = interpolate_line(LineString([start, end]), t)
        pts.append(pt)
    return np.array(pts)


rotate_pts_around_pts_center_np shapes (2,3) and (2,2) not aligned: 3 (dim 1) != 2 (dim 0) 
def rotate_pts_around_pts_center_np(pts_np, angle_deg):
  center_np = np.mean(pts_np, axis=0)
  new_pts_np = rotate_pts_around_center_np(pts_np, angle_deg, center_np)
  return new_pts_np


scale_pts_around_centroid_np translate_pts_np() got an unexpected keyword argument 'delta' 
def scale_pts_around_centroid_np(pts_np, scale_x, scale_y):
    centroid_np = np.mean(pts_np, axis=0)
    

/usr/local/lib/python3.7/dist-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")


In [ ]:
print('cushman | Hierarchical Code-Gen | Flat Prompt')

f_gens_cushman_hc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=True, query_kwargs={'engine': 'code-cushman-001'})

results_cushman_hc_fp = eval_problems(all_problems, f_gens_cushman_hc_fp)
failures_cushman_hc_fp = [r for r in results_cushman_hc_fp if not r['success']]
for failure in failures_cushman_hc_fp:
  print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

cushman | Hierarchical Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

name 'Ellipse' is not defined
obj_i.shape[0] != 4
Success rate: 21/37 = 0.57
bbox_xyxy_contains_pt bbox_xyxy_contains_pt() takes 1 positional argument but 2 were given 
def bbox_xyxy_contains_pt(bbox_xyxy):
    xmin, ymin, xmax, ymax = bbox_xyxy
    return xmin <= pt_np[0] <= xmax and ymin <= pt_np[1] <= ymax


interpolate_pts_np  
def interpolate_pts_np(start, end, n):
  pts = []
  for i in range(n):
    t = i / n
    pt = interpolate_line(LineString([start, end]), t)
    pts.append(pt)
  return np.array(pts)


rotate_pts_around_pts_center_np 'numpy.ndarray' object has no attribute 'is_empty' 
def rotate_pts_around_pts_center_np(pts_np, angle_deg):
  pts_center_np = np.mean(pts_np, axis=0)
  pts_np_rotated = rotate(pts_np, angle_deg, origin=pts_center_np)
  return pts_np_rotated


scale_pts_around_centroid_np 'numpy.ndarray' object has no attribute 'is_empty' 
def scale_pts_around_centroid_np(pts_np, scale_x, scale_y):
    centroid_np = np.mean(pts_np, axis=0)
    new_pts_np = pts_np 

<string>:6: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
/usr/local/lib/python3.7/dist-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")


In [ ]:
print('cushman | Flat Code-Gen | Hierarchical Prompt')

f_gens_cushman_fc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=False, query_kwargs={'engine': 'code-cushman-001'})

results_cushman_fc_hp = eval_problems(all_problems, f_gens_cushman_fc_hp)

for r in results_cushman_fc_hp:
  print(int(r['success']))

# failures_cushman_fc_hp = [r for r in results_cushman_fc_hp if not r['success']]
# for failure in failures_cushman_fc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

cushman | Flat Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

B_mat is not stable.
B_mat is not stable.
B_mat is not stable.
B_mat is not stable.
B_mat is not stable.
Success rate: 20/37 = 0.54
1
1
1
1
1
1
1
0
1
1
1
1
0
0
1
0
0
0
1
0
0
0
0
1
1
1
1
0
1
0
0
0
0
0
1
0
1


In [ ]:
print('cushman | Flat Code-Gen | Flat Prompt')

f_gens_cushman_fc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=False, query_kwargs={'engine': 'code-cushman-001'})

results_cushman_fc_fp = eval_problems(all_problems, f_gens_cushman_fc_fp)

for r in results_cushman_fc_fp:
  print(int(r['success']))

# failures_cushman_fc_fp = [r for r in results_cushman_fc_fp if not r['success']]
# for failure in failures_cushman_fc_fp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

cushman | Flat Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

Success rate: 22/37 = 0.59
1
1
1
1
1
1
1
0
1
0
1
1
0
0
1
0
1
0
1
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
1
1
1


/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2342: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y = asanyarray(b)
/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2342: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = asanyarray(b)


### text-davinci-002

In [ ]:
print('daVinci | Hierarchical Code-Gen | Hierarchical Prompt')

f_gens_davinci_hc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=True, query_kwargs={'engine': 'text-davinci-002'})

results_davinci_hc_hp = eval_problems(all_problems, f_gens_davinci_hc_hp)
failures_davinci_hc_hp = [r for r in results_davinci_hc_hp if not r['success']]
for failure in failures_davinci_hc_hp:
  print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

daVinci | Hierarchical Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

Success rate: 31/37 = 0.84
eval_pts_2d_from_poly_coeffs  
def eval_pts_2d_from_poly_coeffs(poly_x_coeffs, poly_y_coeffs, ts):
    pts_2d_np = np.array([eval_poly(ts, poly_x_coeffs), eval_poly(ts, poly_y_coeffs)]).T
    return pts_2d_np


end_effector_impedance_control shapes (6,7) and (6,) not aligned: 7 (dim 1) != 6 (dim 0) 
def end_effector_impedance_control(x_curr, x_goal, x_dot, K_x_mat, D_x_mat, J):
    # compute the error.
    x_err = x_goal - x_curr
    # compute the error velocity.
    x_err_dot = -x_dot
    # compute the impedance control.
    tau = K_x_mat.dot(x_err) + D_x_mat.dot(x_err_dot) + J.dot(x_dot)
    return tau


is_closed_loop_discrete_system_stable operands could not be broadcast together with shapes (5,3) (3,5)  
def is_closed_loop_discrete_system_stable(A_mat, B_mat, K_mat):
    A_cl_mat = A_mat - B_mat * K_mat
    eigen_values = np.linalg.eigvals(A_cl_mat)
    is_stable = np.all(np.abs(eigen_values) < 1)
    return is_stable


get_direction_orthogonal_to_line  

In [ ]:
print('daVinci | Hierarchical Code-Gen | Flat Prompt')

f_gens_davinci_hc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=True, query_kwargs={'engine': 'text-davinci-002'})

results_davinci_hc_fp = eval_problems(all_problems, f_gens_davinci_hc_fp)
failures_davinci_hc_fp = [r for r in results_davinci_hc_fp if not r['success']]
for failure in failures_davinci_hc_fp:
  print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

daVinci | Hierarchical Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

pd_control

def pd_control(x_curr, x_goal, x_dot, Kp, Kv):
    # TODO: Implement PD control.
    # Hint: Use the eval_line function.
    # Hint: Use the get_total function.
    # Hint: Use the get_pt_to_the_left function.
    # Hint: Use the get_pt_to_the_top function.
    # Hint: Use the interpolate_line function.
    # Hint: Use the make_line_by_length function.
    # Hint: Use the make_vertical_line_by_length function.
    # Hint: Use the rotate function.
    # Hint: Use the scale function.
    # Hint: Use the translate function.
    # Hint: Use the np.array function.
    # Hint: Use the np.sum function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.inv function.
    # Hint: Use the np.dot function.
    # Hint: Use the np.array_equal function.
    # Hint: Use the np.array_equal function.
    # Hint: Use the np.array_equal function.
    # Hint: Use the np.array_equal function.
    # Hint: Use the np.array_equal function.
    # Hint: Use the np.array_equal

In [ ]:
print('daVinci | Flat Code-Gen | Hierarchical Prompt')

f_gens_davinci_fc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=False, query_kwargs={'engine': 'text-davinci-002'})

results_davinci_fc_hp = eval_problems(all_problems, f_gens_davinci_fc_hp)

for r in results_davinci_fc_hp:
  print(int(r['success']))

# failures_davinci_fc_hp = [r for r in results_davinci_fc_hp if not r['success']]
# for failure in failures_davinci_fc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

daVinci | Flat Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

Success rate: 26/37 = 0.70
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
0
0
1
0
1
0
0
1
1
1
1
1
1
1
0
0
1
0
0
1
1


In [ ]:
print('daVinci | Flat Code-Gen | Flat Prompt')

f_gens_davinci_fc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=False, query_kwargs={'engine': 'text-davinci-002'})

results_davinci_fc_fp = eval_problems(all_problems, f_gens_davinci_fc_fp)

for r in results_davinci_fc_fp:
  print(int(r['success']))

# failures_davinci_fc_fp = [r for r in results_davinci_fc_fp if not r['success']]
# for failure in failures_davinci_fc_fp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

daVinci | Flat Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

pd_control

def pd_control(x_curr, x_goal, x_dot, Kp, Kv):
    # TODO: Implement PD control.
    # Hint: Use the eval_line function.
    # Hint: Use the get_total function.
    # Hint: Use the move_pt_left function.
    # Hint: Use the move_pt_up function.
    # Hint: Use the make_line_by_length function.
    # Hint: Use the make_vertical_line_by_length function.
    # Hint: Use the interpolate_line function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
    # Hint: Use the np.linalg.norm function.
  

### text-curie-001

In [ ]:
print('text-curie-001 | Hierarchical Code-Gen | Hierarchical Prompt')

query_kwargs = {'engine': 'text-curie-001', 'frequency_penalty': 0.1}

f_gens_curie_hc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=True, bug_fix=True, query_kwargs=query_kwargs)

results_curie_hc_hp = eval_problems(all_problems, f_gens_curie_hc_hp)
failures_curie_hc_hp = [r for r in results_curie_hc_hp if not r['success']]
# for failure in failures_curie_hc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

text-curie-001 | Hierarchical Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

get_closest_idx
#
def get_closest_idx(points_np, point_np):

    idx = 0
    for point in points_np:
        if point.x == point_np.x and point.y == point_np.y:
            idx = idx + 1
        elif point.x != point_np.x and point.y != point_np.y:
            idx = idx + 1
        elif point.x == point_np.x and point.y != point_np.y and point.z == point_np.z:
            idx = idx + 1
        elif point.x != point_np.x and point.y != point_np.y and point.z != point_np.z:
            # out of range
            # print("Incorrect coordinates")
            # break
            # else:
            # print("Point %s at (%s, %s) has coordinates (%s)".format(point.x, point.y, point.z, point.str))
            # idx = idx + 1
            # return idx
unexpected EOF while parsing (<string>, line 19)
get_bbox_xyxy_area
import numpy as np

def get_bbox_xyxy_area(bbox_xyxy):
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_xyxy
    return np.array(xmin*xmax+ymin*ymax+zmin*zmax)

interpolate_pts_np
imp

In [ ]:
print('text-curie-001 | Hierarchical Code-Gen | Flat Prompt')

query_kwargs = {'engine': 'text-curie-001', 'frequency_penalty': 0.1}

f_gens_curie_hc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=True, bug_fix=True, query_kwargs=query_kwargs)

results_curie_hc_fp = eval_problems(all_problems, f_gens_curie_hc_fp)
failures_curie_hc_fp = [r for r in results_curie_hc_fp if not r['success']]
# for failure in failures_curie_hc_fp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

text-curie-001 | Hierarchical Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

get_right_most_idx
import numpy as np

def get_right_most_idx(points_np, axis=1):
  idx = np.argmax(points_np, axis=axis)
  return idx

get_bbox_xyxy_area
import numpy as np

def get_bbox_xyxy_area(bbox_xyxy):
    xmin, xmax, ymin, ymax = bbox_xyxy.minmax()
    return np.array([xmin * np.size(bbox_xyxy), xmax * np.size(bbox_xyxy), ymin * np.size(bbox_xyxy), ymax * np.size(bbox_xyxy)])

interpolate_pts_np
import numpy as np

def interpolate_pts_np(start, end, n):
  pts = np.linspace(start, end, n)
  return pts

reverse_pts
import numpy as np


def reverse_pts(pts_np):
    return pts_np[::-1, :]


def get_pts_from_file(file_path):
    pts_np = np.loadtxt(file_path, delimiter=',')
    return pts_np


def save_pts_to_file(pts_np, file_path):
    np.savetxt(file_path, pts_np, delimiter=',')


def main():
    file_path = './data/pts.csv'
    pts_np = get_pts_from_file(file_path)
    pts_np = reverse_pts(pts_np)
    save_pts_to_file(pts_np, file_path)


if __name__ == '__main__':
    main()



<string>:6: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2342: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y = asanyarray(b)
/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2342: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=ob

In [ ]:
print('text-curie-001 | Flat Code-Gen | Hierarchical Prompt')

query_kwargs = {'engine': 'text-curie-001', 'frequency_penalty': 0.1}

f_gens_curie_fc_hp = solve_problems(all_problems, prompt_f_gen, context_vars, recurse=False, bug_fix=True, query_kwargs=query_kwargs)

results_curie_fc_hp = eval_problems(all_problems, f_gens_curie_fc_hp)
failures_curie_fc_hp = [r for r in results_curie_fc_hp if not r['success']]
# for failure in failures_curie_fc_hp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

text-curie-001 | Flat Code-Gen | Hierarchical Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

get_bbox_xyxy_area
import numpy as np

def get_bbox_xyxy_area(bbox_xyxy):
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_xyxy
    return np.array(xmin*xmax+ymin*ymax+zmin*zmax)

interpolate_pts_np
import numpy as np

def interpolate_pts_np(start, end, n):
    pts = np.interp(np.linspace(0, 1, n), [0, 1], [start, end])
    return pts

reverse_pts
import numpy as np


def reverse_pts(pts_np):
    pts = pts_np[::-1]
    return pts


def main():
    pts = np.array([[1, 2], [3, 4], [5, 6]])
    print(pts)
    print(reverse_pts(pts))


if __name__ == '__main__':
    main()

normalize_vector
import numpy as np


def normalize_vector(vector):
    return np.array(vector) / np.sum(vector)  # noqa

evaluate_pts_2d_from_poly_coeffs
import numpy as np


def evaluate_pts_2d_from_poly_coeffs(poly_x_coeffs, poly_y_coeffs, ts):
    x_coords, y_coords = poly_x_coeffs.shape
    x_intercept, y_intercept = poly_y_coeffs.shape

    # get the 2D points
    pts = pts_2d_np(poly_x_coeffs, poly_y_coeffs, ts)

  

In [ ]:
print('text-curie-001 | Flat Code-Gen | Flat Prompt')

query_kwargs = {'engine': 'text-curie-001', 'frequency_penalty': 0.1}

f_gens_curie_fc_fp = solve_problems(all_problems, prompt_f_gen_flat, context_vars, recurse=False, bug_fix=True, query_kwargs=query_kwargs)

results_curie_fc_fp = eval_problems(all_problems, f_gens_curie_fc_fp)
failures_curie_fc_fp = [r for r in results_curie_fc_fp if not r['success']]
# for failure in failures_curie_fc_fp:
#   print(failure['f_name'], failure['info'], failure['f_gen']['f_src'])

text-curie-001 | Flat Code-Gen | Flat Prompt


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

get_right_most_idx
import numpy as np

def get_right_most_idx(points_np, axis=1):
  idx = np.argmax(points_np, axis=axis)
  return idx

get_bbox_xyxy_area
import numpy as np

def get_bbox_xyxy_area(bbox_xyxy):
    xmin, xmax, ymin, ymax = bbox_xyxy.minmax()
    return np.array([xmin * np.size(bbox_xyxy), xmax * np.size(bbox_xyxy), ymin * np.size(bbox_xyxy), ymax * np.size(bbox_xyxy)])

interpolate_pts_np
import numpy as np

def interpolate_pts_np(start, end, n):
  pts = np.linspace(start, end, n)
  return pts

reverse_pts
import numpy as np


def reverse_pts(pts_np):
    return pts_np[::-1, :]


def get_pts_from_file(file_path):
    pts_np = np.loadtxt(file_path, delimiter=',')
    return pts_np


def save_pts_to_file(pts_np, file_path):
    np.savetxt(file_path, pts_np, delimiter=',')


def main():
    file_path = './data/pts.csv'
    pts_np = get_pts_from_file(file_path)
    pts_np = reverse_pts(pts_np)
    save_pts_to_file(pts_np, file_path)


if __name__ == '__main__':
    main()



/usr/local/lib/python3.7/dist-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
